In [ ]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

# resolving "No inf checks were recorded for this optimizer." issue
!pip uninstall torch -y
!pip install torch==2.1

!pip install torch torchvision torchaudio --upgrade

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers

model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto", # automatically figures out how to best use CPU + GPU for loading model
                                             trust_remote_code=False, # prevents running custom model files on your machine
                                             revision="main") # which version of model to use in repo
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

model.eval() # model in evaluation mode (dropout modules are deactivated)

# craft prompt
comment = "Great content, please can you elaborate on what datascience is, thank you!"
prompt=f'''[INST] {comment} [/INST]'''

# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

# Set model to evaluation mode
model.eval()  # Dropout and batchnorm layers are deactivated

# Craft a prompt
comment = "What are the latest trends in datascience and artificial intelligence?"
prompt = f'''[INST]"{comment}"[/INST]'''

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=100)

# Decode and print output
print(tokenizer.batch_decode(outputs)[0])


# Switch model to evaluation mode
model.eval()  # Ensures consistent predictions during inference

# Create a prompt
question = "How can I build a recommendation system for e-commerce?"
prompt = f'''[INST] Provide a step-by-step guide to: "{question}" [/INST]'''

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=150)

# Decode and print output
print(tokenizer.batch_decode(outputs)[0])

# Switch model to evaluation mode
model.eval()  # Ensures consistent predictions during inference

# Create a prompt
question = "How do algorithms optimize the training of large language models?"
prompt = f'''[INST] "{question}" [/INST]'''

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=150)

# Decode and print output
print(tokenizer.batch_decode(outputs)[0])

intstructions_string = f"""ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. \
ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Please respond to the following comment.
"""

prompt_template = lambda comment: f'''[INST] {intstructions_string} \n{comment} \n[/INST]'''

prompt = prompt_template(comment)
print(prompt)

# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

model.train() # model in training mode (dropout modules are activated)

# enable gradient check pointing
model.gradient_checkpointing_enable()

# enable quantized training
model = prepare_model_for_kbit_training(model)

# LoRA config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model = get_peft_model(model, config)

# trainable parameter count
model.print_trainable_parameters()

# load dataset
data = load_dataset("shawhin/shawgpt-youtube-comments")

# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["example"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

# tokenize training and validation datasets
tokenized_data = data.map(tokenize_function, batched=True)

# setting pad token
tokenizer.pad_token = tokenizer.eos_token
# data collator
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)

# hyperparameters
lr = 2e-4
batch_size = 4
num_epochs = 10

# define training arguments
training_args = transformers.TrainingArguments(
    output_dir= "shawgpt-ft",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",

)

# configure trainer
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    args=training_args,
    data_collator=data_collator
)


# train model
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

# renable warnings
model.config.use_cache = True

from huggingface_hub import notebook_login
notebook_login()

# # option 2: key login
# from huggingface_hub import login
# write_key = 'hf_' # paste token here
# login(write_key)

hf_name = 'shadow0017' # your hf username or org name
model_id = hf_name + "/" + "shawgpt-ft"

intstructions_string = f"""ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. \
ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Please respond to the following comment.
"""
prompt_template = lambda comment: f'''[INST] {intstructions_string} \n{comment} \n[/INST]'''

comment = "Great content, please can you elaborate on what datascience is, thank you!"

prompt = prompt_template(comment)
print(prompt)

model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

comment = "Great content, please can you elaborate on what datascience is, thank you!"
prompt = prompt_template(comment)

model.eval()
inputs = tokenizer(prompt, return_tensors="pt")

outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)
print(tokenizer.batch_decode(outputs)[0])


# Define comment and use the prompt template
comment = "What are the latest trends in datascience and artificial intelligence?"
prompt = prompt_template(comment)

# Set model to evaluation mode
model.eval()  # Deactivates dropout and other non-deterministic layers

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

# Decode and print output
print(tokenizer.batch_decode(outputs)[0])


# Define comment and use the prompt template
comment = "How can I build a recommendation system for e-commerce?"
prompt = prompt_template(comment)

# Set model to evaluation mode
model.eval()  # Deactivates dropout and other non-deterministic layers

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

# Decode and print output
print(tokenizer.batch_decode(outputs)[0])

# Define comment and use the prompt template
comment = "How do algorithms optimize the training of large language models?"
prompt = prompt_template(comment)

# Set model to evaluation mode
model.eval()  # Deactivates dropout and other non-deterministic layers

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

# Decode and print output
print(tokenizer.batch_decode(outputs)[0])


# Define comment and apply the prompt template
comment = "How do algorithms optimize the training of large language models?"
prompt = prompt_template(comment)

# Set model to evaluation mode
model.eval()

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# Generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

# Decode and print output
print(tokenizer.batch_decode(outputs)[0])
